# Salary Prediction from LinkedIn Job Postings - Train XGBoost Model

In [1]:
import xgboost as xgb
import pandas as pd, numpy as np
import salary
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
from sklearn.impute import SimpleImputer

## Train & Evaluate Model

In [2]:
(X_train, y_train) = salary.get_train_dataset()
(X_test, y_test) = salary.get_test_dataset()


### Target normal salaries

In [3]:
model = make_pipeline(
    ColumnTransformer(
        transformers=[
            ('target_encoder', TargetEncoder(), ['norm_title', 'location_state', 'company_industries']),
            ('experience_level', salary.experience_level_encoder, ['formatted_experience_level']),
            ('work_type', salary.work_type_encoder, ['formatted_work_type']),
            ('remote_allowed', 'passthrough', ['remote_allowed']),
            ('company_employee_count', SimpleImputer(strategy='median'), ['company_employee_count']),
        ],
        remainder='drop'
    ),
    StandardScaler(),
    xgb.XGBRegressor(
        n_estimators=1000,
        max_depth=5,
        learning_rate=0.1,
        verbosity=2,
        objective='reg:squarederror',
        subsample=0.8,
        random_state=42
    )
).fit(X_train, y_train)

salary.evaluate_train_predictions(model.predict(X_train))

[00:00:23] INFO: /Users/runner/work/xgboost/xgboost/src/data/iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (27885, 7, 195195).
Train R2: 0.7140
Train RMSE: 32900.1542
Train MAE: 21767.4781


In [4]:
salary.evaluate_test_predictions(model.predict(X_test))

Test R2: 0.4802
Test RMSE: 42032.4344
Test MAE: 27563.9645
On average, our predicted salaries are $27563.96 off the true salaries
This is 36.52% better than a naive global mean


### Target log salaries


In [5]:
model_log = make_pipeline(
    ColumnTransformer(
        transformers=[
            ('target_encoder', TargetEncoder(), ['norm_title', 'location_state', 'company_industries']),
            ('experience_level', salary.experience_level_encoder, ['formatted_experience_level']),
            ('work_type', salary.work_type_encoder, ['formatted_work_type']),
            ('remote_allowed', 'passthrough', ['remote_allowed']),
            ('company_employee_count', SimpleImputer(strategy='median'), ['company_employee_count']),
        ],
        remainder='drop'
    ),
    StandardScaler(),
    TransformedTargetRegressor(
        regressor=xgb.XGBRegressor(
            n_estimators=1000,
            max_depth=5,
            learning_rate=0.1,
            verbosity=2,
            objective='reg:squarederror',
            subsample=0.8,
            random_state=42
        ),
        func=np.log10,
        inverse_func=lambda x: 10**x
    )
).fit(X_train, y_train)

salary.evaluate_train_predictions(model.predict(X_train))

[00:00:24] INFO: /Users/runner/work/xgboost/xgboost/src/data/iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (27885, 7, 195195).
Train R2: 0.7140
Train RMSE: 32900.1542
Train MAE: 21767.4781


In [6]:
salary.evaluate_test_predictions(model_log.predict(X_test))

Test R2: 0.4901
Test RMSE: 41629.5911
Test MAE: 26117.6546
On average, our predicted salaries are $26117.65 off the true salaries
This is 39.85% better than a naive global mean


### Feature Importance

In [10]:
feature_names = model[:-1].get_feature_names_out(salary.df_X.columns)

def get_important_features(coef, n=10):
    idxs = np.flip(np.argsort(np.abs(coef))[-n:])
    return pd.DataFrame([feature_names[idxs], coef[idxs]], index=["Feature", "Coefficient"]).transpose()

get_important_features(model[-1].feature_importances_)


,Feature,Coefficient
0,experience_level__formatted_experience_level,0.325434
1,target_encoder__norm_title,0.172653
2,target_encoder__company_industries,0.147325
3,work_type__formatted_work_type,0.096083
4,target_encoder__location_state,0.092207
5,company_employee_count__company_employee_count,0.085692
6,remote_allowed__remote_allowed,0.080606
